# Amplifier

In [1]:
import os
import control
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt

duration = 26.84e-3
delays = np.linspace(0, duration, 300)

os.makedirs('../dataset/amplifier', exist_ok=True)

scope = control.MSOX6004A('172.22.22.30')
fngen = control.FG33250A2('172.22.22.36')

scope.holla(), fngen.holla()

('AGILENT TECHNOLOGIES,MSO-X 4054A,MY57310545,07.20.2017102615\n',
 'Agilent Technologies,33220A,MY43003480,1.05-1.05-22-1\n')

## Measurement

Insert the external trigger signal to the back of the Agilent 33220A function generator and connect its output with the external trigger input of the oscilloscope. Let the oscilloscope trigger on falling edge and set the time scale to 20 us. The function generator should be configured to output pulse signals with period of 30 ms.

The output of the horizontal DDS is connected to the horizontal amplifier. The DDS driver is configured with inverse sinc filter turned off in the subsequent experiments.

The output of the horizontal amplifier is connected to the following attentuators to not destory the oscillscope:

1. 1 dB
2. 3 dB
3. 6 dB
4. 10 dB
5. 10 dB

With the damping power increasing towards the oscilloscope such that the heat is distributed more evenly.

In [2]:
control.aod_h(frequency=[80e6, 120e6], amplitude=np.ones(1024), duration=duration)

data = []

for t in delays:
    fngen.width(t)
    
    data.append((t, scope.capture()))

for (t, df) in data:
    df.to_hdf('../dataset/amplifier/horizontal.h5', str(t), format='table', complevel=9)

Now connect the output of the horizontal DDS to the input of the vertical amplifier. Then connect the output of the vertical amplifier through the same attentuator configurator as before with the oscilloscope.

In [3]:
control.aod_h(frequency=[80e6, 120e6], amplitude=np.ones(1024), duration=duration)

data = []

for t in delays:
    fngen.width(t)
    
    data.append((t, scope.capture()))

for (t, df) in data:
    df.to_hdf('../dataset/amplifier/vertical.h5', str(t), format='table', complevel=9)

Now connect the vertical DDS with the oscillscope output. Check again that the DDS driver has the inverse sinc filter turned off.

## Evaluation

We are interested in the signal amplitude in dependence of the frequency. Thus we will do a FFT on each segment to identify the dominant frequency and take the maximum voltage as amplitude. Then we compare the different configuration.

In [4]:
def fft(df):
    T = df.time.values[1] - df.time.values[0]
    N = df.voltage.size
    
    A = np.abs(np.fft.fft(df.voltage)) / N
    f = np.linspace(0, 1/T, N)

    return f[:N // 2], A[:N // 2]